In [3]:
import os
import glob
import pandas as pd
import rasterio
import numpy as np




def preparer_donnees_climat_rapide():
    """
    Version optimisée - devrait prendre 3-6 minutes au lieu de 30
    """
    base_path = r"C:\Users\hp\Desktop\TPs\DataMining\DATA"
    
    # Charger la grille
    df_grille = pd.read_csv("points.csv")
    print(f"📍 Grille: {len(df_grille)} points")
    
    # DataFrame résultat
    df_complet = df_grille[['longitude', 'latitude']].copy()
    
    # Traitement optimisé
    dossiers_climat = {
        'Precipitation': 'prec',
        'Tmin': 'tmin', 
        'Tmax': 'tmax'
    }
    
    for dossier, prefixe in dossiers_climat.items():
        chemin_dossier = os.path.join(base_path, "Climate", dossier)
        fichiers_tif = glob.glob(os.path.join(chemin_dossier, "*.tif"))
        
        print(f"\n📁 {dossier}: {len(fichiers_tif)} fichiers")
        
        for tif_path in sorted(fichiers_tif):
            nom_fichier = os.path.basename(tif_path)
            mois = nom_fichier.split('_')[-1].replace('.tif', '')
            nom_colonne = f"{prefixe}_{mois}"
            
            print(f"   ⚡ {mois}...", end=" ")
            
            # EXTRACTION VECTORISÉE (RAPIDE)
            valeurs = extraction_vectorisee(tif_path, df_grille)
            df_complet[nom_colonne] = valeurs
            
            valides = np.sum(~np.isnan(valeurs))
            print(f"✅ ({valides} valides)")
    
    # Sauvegarde
    output_csv = os.path.join(base_path, "donnees_climat_1km_rapide.csv")
    df_complet.to_csv(output_csv, index=False)
    
    return df_complet

# Version vectorisée à implémenter
def extraction_vectorisee(tif_path, df_points):
    """
    Extraction 10x plus rapide avec opérations vectorisées
    """
    with rasterio.open(tif_path) as src:
        # Conversion de TOUTES les coordonnées en une fois
        rows, cols = rasterio.transform.rowcol(
            src.transform, 
            df_points['longitude'].values, 
            df_points['latitude'].values
        )
        
        # Masque des points valides
        mask = (rows >= 0) & (rows < src.height) & (cols >= 0) & (cols < src.width)
        
        # Initialisation du tableau de résultats
        valeurs = np.full(len(df_points), np.nan)
        
        if np.any(mask):
            # Extraction des valeurs valides
            data = src.read(1)
            valeurs_valides = data[rows[mask], cols[mask]]
            
            # Application du masque
            valeurs[mask] = valeurs_valides
            
            # Gestion NoData
            valeurs[valeurs == src.nodata] = np.nan
    
    return valeurs

# Essayez cette version plus rapide
donnees_rapides = preparer_donnees_climat_rapide()

📍 Grille: 2481850 points

📁 Precipitation: 12 fichiers
   ⚡ 2024-01... ✅ (2481827 valides)
   ⚡ 2024-02... ✅ (2481827 valides)
   ⚡ 2024-03... ✅ (2481827 valides)
   ⚡ 2024-04... ✅ (2481827 valides)
   ⚡ 2024-05... ✅ (2481827 valides)
   ⚡ 2024-06... ✅ (2481827 valides)
   ⚡ 2024-07... ✅ (2481827 valides)
   ⚡ 2024-08... ✅ (2481827 valides)
   ⚡ 2024-09... ✅ (2481827 valides)
   ⚡ 2024-10... ✅ (2481827 valides)
   ⚡ 2024-11... ✅ (2481827 valides)
   ⚡ 2024-12... ✅ (2481827 valides)

📁 Tmin: 12 fichiers
   ⚡ 2024-01... ✅ (2481827 valides)
   ⚡ 2024-02... ✅ (2481827 valides)
   ⚡ 2024-03... ✅ (2481827 valides)
   ⚡ 2024-04... ✅ (2481827 valides)
   ⚡ 2024-05... ✅ (2481827 valides)
   ⚡ 2024-06... ✅ (2481827 valides)
   ⚡ 2024-07... ✅ (2481827 valides)
   ⚡ 2024-08... ✅ (2481827 valides)
   ⚡ 2024-09... ✅ (2481827 valides)
   ⚡ 2024-10... ✅ (2481827 valides)
   ⚡ 2024-11... ✅ (2481827 valides)
   ⚡ 2024-12... ✅ (2481827 valides)

📁 Tmax: 12 fichiers
   ⚡ 2024-01... ✅ (2481827 valides)
   

In [8]:
import pandas as pd

df=pd.read_csv(r"C:\Users\hp\Desktop\TPs\DataMining\DATA\donnees_climat_1km_rapide.csv")
df.head()

def transformer_mois_en_saisons(df_climat):
    """
    Remplace les 12 colonnes mensuelles par 4 colonnes saisonnières pour chaque variable climatique
    """
    print("🔄 TRANSFORMATION MOIS → SAISONS")
    print("=" * 50)
    
    # Créer un nouveau DataFrame avec les coordonnées
    df_saisons = df_climat[['longitude', 'latitude']].copy()
    
    # Définition des saisons (3 mois chacune)
    saisons = {
        'hiver': ['01', '02', '12'],    # Décembre, Janvier, Février
        'printemps': ['03', '04', '05'], # Mars, Avril, Mai
        'ete': ['06', '07', '08'],      # Juin, Juillet, Août  
        'automne': ['09', '10', '11']   # Septembre, Octobre, Novembre
    }
    
    # Variables climatiques à traiter
    variables = ['prec', 'tmin', 'tmax']
    
    for variable in variables:
        print(f"\n🌡️  Traitement de: {variable}")
        
        # Trouver toutes les colonnes pour cette variable
        colonnes_variable = [col for col in df_climat.columns if col.startswith(f"{variable}_")]
        print(f"   📅 Colonnes mensuelles: {len(colonnes_variable)}")
        
        for saison, mois_list in saisons.items():
            # Colonnes correspondant à cette saison
            colonnes_saison = [col for col in colonnes_variable if any(f"-{m}" in col for m in mois_list)]
            
            if len(colonnes_saison) == 3:  # Doit avoir exactement 3 mois
                print(f"   🍂 {saison}: {colonnes_saison}")
                
                # Calculer la MOYENNE des 3 mois
                df_saisons[f'{variable}_{saison}'] = df_climat[colonnes_saison].mean(axis=1)
            else:
                print(f"   ⚠️  {saison}: {len(colonnes_saison)} mois trouvés (attendu: 3)")
    
    # ANALYSE FINALE
    colonnes_originales = len([c for c in df_climat.columns if any(v in c for v in ['prec', 'tmin', 'tmax'])])
    colonnes_saisonnières = len([c for c in df_saisons.columns if any(v in c for v in ['prec', 'tmin', 'tmax'])])
    
    print(f"\n✅ TRANSFORMATION TERMINÉE:")
    print(f"   📊 Colonnes originales (mensuelles): {colonnes_originales}")
    print(f"   📈 Colonnes saisonnières: {colonnes_saisonnières}")
    print(f"   📉 Réduction: {colonnes_originales} → {colonnes_saisonnières} colonnes")
    
    return df_saisons

# Application
df_saisons = transformer_mois_en_saisons(df)
df_saisons.head()

df_saisons.to_csv(r"C:\Users\hp\Desktop\TPs\DataMining\DATA\climat_saisons.csv", index=False)


🔄 TRANSFORMATION MOIS → SAISONS

🌡️  Traitement de: prec
   📅 Colonnes mensuelles: 12
   🍂 hiver: ['prec_2024-01', 'prec_2024-02', 'prec_2024-12']
   🍂 printemps: ['prec_2024-03', 'prec_2024-04', 'prec_2024-05']
   🍂 ete: ['prec_2024-06', 'prec_2024-07', 'prec_2024-08']
   🍂 automne: ['prec_2024-09', 'prec_2024-10', 'prec_2024-11']

🌡️  Traitement de: tmin
   📅 Colonnes mensuelles: 12
   🍂 hiver: ['tmin_2024-01', 'tmin_2024-02', 'tmin_2024-12']
   🍂 printemps: ['tmin_2024-03', 'tmin_2024-04', 'tmin_2024-05']
   🍂 ete: ['tmin_2024-06', 'tmin_2024-07', 'tmin_2024-08']
   🍂 automne: ['tmin_2024-09', 'tmin_2024-10', 'tmin_2024-11']

🌡️  Traitement de: tmax
   📅 Colonnes mensuelles: 12
   🍂 hiver: ['tmax_2024-01', 'tmax_2024-02', 'tmax_2024-12']
   🍂 printemps: ['tmax_2024-03', 'tmax_2024-04', 'tmax_2024-05']
   🍂 ete: ['tmax_2024-06', 'tmax_2024-07', 'tmax_2024-08']
   🍂 automne: ['tmax_2024-09', 'tmax_2024-10', 'tmax_2024-11']

✅ TRANSFORMATION TERMINÉE:
   📊 Colonnes originales (mensuell

In [3]:
import pandas as pd

df_climat = pd.read_csv(r"C:\Users\hp\Desktop\TPs\DataMining\csv_final\climat.csv")

print("Arrondissement des coordonnées à 3 décimales...")

df_climat['latitude'] = df_climat['latitude'].round(3)
df_climat['longitude'] = df_climat['longitude'].round(3)

print("Recherche des lignes duplicates...")
masque_doublons = df_climat.duplicated(subset=['latitude', 'longitude'], keep=False)

df_doublons = df_climat[masque_doublons].copy()
df_climat.to_csv(r"C:\Users\hp\Desktop\TPs\DataMining\csv_final\climat.csv", index=False)

        # Résumé
if df_doublons.empty:
            print("Bonne nouvelle : Aucune ligne duplicate trouvée après arrondissement.")
else:
            print(f"Trouvé {len(df_doublons)} lignes qui sont des doublons.")
            
            # Trier pour voir les groupes de doublons ensemble (optionnel)
            df_doublons.sort_values(by=['lat_round', 'lon_round'], inplace=True)

Arrondissement des coordonnées à 3 décimales...
Recherche des lignes duplicates...
Bonne nouvelle : Aucune ligne duplicate trouvée après arrondissement.
